# Image Alignment (Feature Based) using OpenCV

In many Computer Vision applications, we often need to identify interesting stable points in an image. 
These points are called keypoints or feature points. 
There are several keypoint detectors implemented in OpenCV ( e.g. SIFT, SURF, and ORB).

ORB stands for Oriented FAST and Rotated BRIEF.

# A feature point detector has two parts :-

# Locator: 

This identifies points on the image that are stable under image transformations like translation, scale and rotation.
The locator finds the x, y coordinates of such points. The locator used by the ORB detector is called FAST.

# Descriptor:

The locator in the above step only tells us where the interesting points are.
The second part of the feature detector is the descriptor which encodes the appearance of the point 
Descriptor can tell one feature point from the other. 
Ideally, the same physical point in two images should have the same descriptor. 
ORB uses a modified version of the feature descriptor called BRISK.

The Homography that relates the two images can be calculated only if we know corresponding features in the two images. 
So a matching algorithm is used to find which features in one image match features in the other image. 
The descriptor of every feature in one image is compared to the descriptor of every feature in the second image to find good matches.


# Homography:

Two images of a scene are related by a homography under two conditions.

1. The two images are that of a plane (e.g. sheet of paper, credit card etc.).
2. The two images were acquired by rotating the camera about its optical axis. We take such images while generating panoramas.

A homography is nothing but a 3×3 matrix .

![title](homography.png)

Let (x1,y1) be a point in the first image and (x2,y2) be the coordinates of the same physical point in the second image.
Then, the Homography H relates them in the following way :-

![title](homography2.png)

If we knew the homography, we could apply it to all the pixels of one image to obtain a warped image that is aligned with the second image.

In [1]:
import cv2
import numpy as np


MAX_FEATURES = 500
GOOD_MATCH_PERCENT = 0.15
 
 
def alignImages(im1, im2):
 
  # Convert images to grayscale
  im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
  im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
   
  # Detect ORB features and compute descriptors.
  orb = cv2.ORB_create(MAX_FEATURES)
  keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
  keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)
   
  # Match features.
  matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
  matches = matcher.match(descriptors1, descriptors2, None)
   
  # Sort matches by score
  matches.sort(key=lambda x: x.distance, reverse=False)
 
  # Remove not so good matches
  numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
  matches = matches[:numGoodMatches]
 
  # Draw top matches
  imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
  cv2.imwrite("matches.jpg", imMatches)
   
  # Extract location of good matches
  points1 = np.zeros((len(matches), 2), dtype=np.float32)
  points2 = np.zeros((len(matches), 2), dtype=np.float32)
 
  for i, match in enumerate(matches):
    points1[i, :] = keypoints1[match.queryIdx].pt
    points2[i, :] = keypoints2[match.trainIdx].pt
   
  # Find homography
  h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)
 
  # Use homography
  height, width, channels = im2.shape
  im1Reg = cv2.warpPerspective(im1, h, (width, height))
   
  return im1Reg, h
 
 
if __name__ == '__main__':
   
  # Read reference image
  refFilename = "inter_doc.png"
  print("Reading reference image : ", refFilename)
    
  #Using the inbuilt cv2 function to read the image.  
  imReference = cv2.imread(refFilename, cv2.IMREAD_COLOR)
 
  # Read image to be aligned
  imFilename = "inter_doc_totrot.jpg"
  print("Reading image to align : ", imFilename);
    
  #Using the inbuilt cv2 function to read the image.  
  im = cv2.imread(imFilename, cv2.IMREAD_COLOR)
   
  print("Aligning images ...")
  # Registered image will be resotred in imReg. 
  # The estimated homography will be stored in h. 
  imReg, h = alignImages(im, imReference)
   
  # Write aligned image to disk. 
  outFilename = "aligned.jpg"
  print("Saving aligned image : ", outFilename);

  #Using cv2 inbuilt function to save the output image  
  cv2.imwrite(outFilename, imReg)
 
  # Print estimated homography
  print("Estimated homography : \n",  h)

Reading reference image :  inter_doc.png
Reading image to align :  inter_doc_totrot.jpg
Aligning images ...
Saving aligned image :  aligned.jpg
Estimated homography : 
 [[-1.00348921e+00  2.12942270e-03  7.55446845e+02]
 [-7.18917806e-04 -1.00032145e+00  9.71910836e+02]
 [ 1.32739469e-06  3.96730867e-06  1.00000000e+00]]


# Steps for Feature Based Image Alignment :-

# Read Images :

We first read the reference image (or the template image) and the image we want to align to this template in Lines 56-65 in the Python code.

# Detect Features:

We then detect ORB features in the two images. Although we need only 4 features to compute the homography, typically hundreds of features are detected in the two images. We control the number of features using the parameter MAX_FEATURES in the Python Code.Lines 16-19 in the Python code detect features and compute the descriptors using detectAndCompute.

# Match Features:

 Lines 21-34 in Python we find the matching features in the two images, sort them by goodness of match and keep only a small percentage of original matches. We finally display the good matches on the images and write the file to disk for visual inspection. We use the hamming distance as a measure of similarity between two feature descriptors. The matched features are shown in the figure below by drawing a line connecting them.

# Calculate Homography:

 A homography can be computed when we have 4 or more corresponding points in two images. Automatic feature matching explained in the previous section does not always produce 100% accurate matches. It is not uncommon for 20-30% of the matches to be incorrect. Fortunately, the findHomography method utilizes a robust estimation technique called Random Sample Consensus (RANSAC) which produces the right result even in the presence of large number of bad matches. Lines 36-45 in Python accomplish this in code.

# Warping image:

Once an accurate homography has been calculated, the transformation can be applied to all pixels in one image to map it to the other image. This is done using the warpPerspective function in OpenCV. This is accomplished in Line 49 in Python.


In [9]:
#Canny Edge Detection

In [35]:
import cv2
import numpy as np

img = cv2.imread('inter_doc.png',0)

canny = cv2.Canny(img, 50, 120)


# scale_percent = 40 # percent of original size
# width = int(canny.shape[1] * scale_percent / 100)
# height = int(canny.shape[0] * scale_percent / 100)
# dim = (width, height)
# resize image
# resized = cv2.resize(canny, dim, interpolation = cv2.INTER_AREA)

_,mask = cv2.threshold(canny,70,255,cv2.THRESH_BINARY_INV)

kernel = np.ones((1,1),np.uint8)
closing = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
erosion = cv2.erode(mask,kernel,iterations = 1)
opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)

cv2.imshow('Canny', closing)
cv2.waitKey(0)

cv2.destroyAllWindows()

In [37]:
blur = cv2.GaussianBlur(img,(1,1),0)
_,image = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
cv2.imshow('OTSU',image)
cv2.waitKey(0)

cv2.destroyAllWindows()

# Morphological Transformations :-

Morphological transformations are some simple operations based on the image shape. It is normally performed on binary images. It needs two inputs, one is our original image, second one is called structuring element or kernel which decides the nature of operation. Two basic morphological operators are Erosion and Dilation. Then its variant forms like Opening, Closing, Gradient etc also comes into play.

In [4]:
import cv2
import numpy as np

#Reading the Image and converting into gray_scale using the 2nd parameter of imread()
img = cv2.imread('inter_doc.png',0)
kernel = np.ones((1,1),np.uint8)

#Applying Dialation and Erosion to remove noise
closing = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)

#Displaying the output until a key is pressed
cv2.imshow('Closing Morphology : ', closing)
cv2.waitKey(0)

#Closing all the open windows
cv2.destroyAllWindows()

# Closing

Closing is reverse of Opening, Dilation followed by Erosion. It is useful in closing small holes inside the foreground objects, or small black points on the object.

![title](closing.png)

# THRESHOLDING :-

There are Two types of Thresholding used in opencv :-
1. Simple Thresholding
2. Adaptive Thresholding

Simple Thresholding - If pixel value is greater than a threshold value, it is assigned one value (may be white), else it is assigned another value (may be black). The function used is cv2.threshold().

Adaptive Thresholding - In simple thresholding we used a global value as threshold value. But it may not be good in all the conditions where image has different lighting conditions in different areas. In that case, we go for adaptive thresholding. In this, the algorithm calculate the threshold for a small regions of the image. So we get different thresholds for different regions of the same image and it gives us better results for images with varying illumination.

Thresholding has a huge influence on the performance of the various processes, because these are based on the data output coming from the thresholding operation. Improper thresholding can cause blotches, streaks or erasures on the document as well as confounding segmentation and recognition tasks. If someone performs optical character recognition (OCR), merges, fractures and other deformations on character shapes are the main reason of performance loss.

Here Thresholding is applied in :-

• Document image analysis (extracting characters, graphical content, musical scores, etc.)

In [5]:

blur = cv2.GaussianBlur(img,(1,1),0)

_,image = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
#_,image = cv2.threshold(blur,200,255,cv2.THRESH_BINARY)
#image = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)

cv2.imshow('original',img)
cv2.imshow('OTSU',image)
cv2.waitKey(0)

cv2.destroyAllWindows()